<a href="https://colab.research.google.com/github/KotaOnishi/enjap/blob/main/uk_jobs_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up

In [ ]:
!pip install gspread oauth2client

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import gspread
from google.auth import default

creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
!pip install requests beautifulsoup4

# Centre People - Job Links Scraping

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd  # CSV出力用
from datetime import datetime  # 日時取得用
from google.colab import files  # ダウンロード用

# ベースURL（ページ番号はiに置き換え）
base_url = "https://www.centrepeople.com/search?page="

# ページ番号の範囲
page_range = range(1, 14)  # 1から13まで

# 結果を格納するリスト
job_links = []

# 1ページ目から13ページ目までループ
for i in page_range:
    # 各ページのURL
    url = base_url + str(i)

    # HTTPリクエストを送る（User-Agentを指定）
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)

    # HTMLを解析
    soup = BeautifulSoup(response.text, "html.parser")

    # すべてのリンクを取得
    all_links = [a.get("href") for a in soup.find_all("a") if a.get("href")]

    # "https://www.centrepeople.com/jobs/xxxxxxx" で 6文字または7文字のIDのURLだけを抽出し、"area" を含むものは除外
    page_job_links = [
        link for link in all_links
        if re.match(r"^https://www\.centrepeople\.com/jobs/.+", link) and "area" not in link
    ]

    # 抽出されたリンクをjob_linksリストに追加
    job_links.extend(page_job_links)

# 現在の日付と時刻を "yyyymmdd-hhmm" の形式で取得
timestamp = datetime.now().strftime("%Y%m%d-%H%M")

# CSVファイル名を指定
output_filename = f"centre_people_job_links_{timestamp}.csv"

# CSVに保存
df = pd.DataFrame(job_links, columns=["Job URL"])
df.to_csv(output_filename, index=False)

# CSVファイルをダウンロード
files.download(output_filename)

# 結果を出力
print(f"抽出された求人URL数: {len(job_links)}")
print(f"CSVファイル '{output_filename}' に保存しました。")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

抽出された求人URL数: 242
CSVファイル 'centre_people_job_links_20250323-1039.csv' に保存しました。


# Centre people - Job Details Scraping


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from google.colab import files  # ファイルアップロード用
from datetime import datetime  # 日時取得用

# CSVファイルをアップロード
uploaded = files.upload()

# CSVファイル名を指定
csv_filename = list(uploaded.keys())[0]

# CSVファイルを読み込んでURLリストを取得
df_urls = pd.read_csv(csv_filename)

# URLリストの取得
job_urls = df_urls.iloc[:, 0].tolist()  # 最初の列にURLがあると仮定

# スクレイピング結果を格納するリスト
job_data_list = []

# 各求人URLに対してスクレイピングを実行
for job_url in job_urls:
    try:
        # HTTPリクエスト
        headers = {"User-Agent": "Mozilla/5.0"}
        response = requests.get(job_url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")

        # 各項目を取得
        job_title = soup.select_one("body > div:nth-of-type(2) > div > div:nth-of-type(1) > div:nth-of-type(1) > h1")
        sector = soup.select_one("body > div:nth-of-type(2) > div > div:nth-of-type(1) > div:nth-of-type(1) > div:nth-of-type(1) > table > tbody > tr:nth-of-type(1) > td")
        job_type = soup.select_one("body > div:nth-of-type(2) > div > div:nth-of-type(1) > div:nth-of-type(1) > div:nth-of-type(1) > table > tbody > tr:nth-of-type(2) > td")
        location = soup.select_one("body > div:nth-of-type(2) > div > div:nth-of-type(1) > div:nth-of-type(1) > div:nth-of-type(1) > table > tbody > tr:nth-of-type(3) > td")
        salary = soup.select_one("body > div:nth-of-type(2) > div > div:nth-of-type(1) > div:nth-of-type(1) > div:nth-of-type(1) > table > tbody > tr:nth-of-type(4) > td")
        benefit = soup.select_one("body > div:nth-of-type(2) > div > div:nth-of-type(1) > div:nth-of-type(1) > div:nth-of-type(1) > table > tbody > tr:nth-of-type(5) > td")
        job_brief = soup.select_one("body > div:nth-of-type(2) > div > div:nth-of-type(1) > div:nth-of-type(1) > div:nth-of-type(2)")
        job_description = soup.select_one("body > div:nth-of-type(2) > div > div:nth-of-type(1) > div:nth-of-type(1) > div:nth-of-type(3)")
        updated = soup.select_one("body > div:nth-of-type(2) > div > div:nth-of-type(1) > div:nth-of-type(1) > div:nth-of-type(4) > div > div:nth-of-type(1)")
        reference = soup.select_one("body > div:nth-of-type(2) > div > div:nth-of-type(1) > div:nth-of-type(1) > div:nth-of-type(4) > div > div:nth-of-type(2)")

        # データを辞書にまとめる
        job_data = {
            "URL": job_url,
            "Job Title": job_title.text.strip() if job_title else "N/A",
            "Sector": sector.text.strip() if sector else "N/A",
            "Type": job_type.text.strip() if job_type else "N/A",
            "Location": location.text.strip() if location else "N/A",
            "Salary": salary.text.strip() if salary else "N/A",
            "Benefit": benefit.text.strip() if benefit else "N/A",
            "Job Brief": job_brief.text.strip() if job_brief else "N/A",
            "Job Description": job_description.text.strip() if job_description else "N/A",
            "Updated": updated.text.strip() if updated else "N/A",
            "Reference": reference.text.strip() if reference else "N/A"
        }

        # リストに追加
        job_data_list.append(job_data)

        print(f"✅ {job_url} のデータ取得完了")

    except Exception as e:
        print(f"⚠️ {job_url} の取得に失敗: {e}")

# pandasのDataFrameに変換
df_jobs = pd.DataFrame(job_data_list)

# 現在の日付と時刻を "yyyymmdd-hhmm" の形式で取得
timestamp = datetime.now().strftime("%Y%m%d-%H%M")

# CSVとして保存
output_path = f"centrepeople_job_details_{timestamp}.csv"
df_jobs.to_csv(output_path, index=False)

print(f"🎉 すべてのデータを {output_path} に保存しました！")

# ダウンロードリンクを表示
files.download(output_path)


Saving centre_people_job_links_20250323-1039.csv to centre_people_job_links_20250323-1039 (2).csv
✅ https://www.centrepeople.com/jobs/RS46401 のデータ取得完了
✅ https://www.centrepeople.com/jobs/CC46409 のデータ取得完了
✅ https://www.centrepeople.com/jobs/TY46405 のデータ取得完了
✅ https://www.centrepeople.com/jobs/YC46396 のデータ取得完了
✅ https://www.centrepeople.com/jobs/MY46368 のデータ取得完了
✅ https://www.centrepeople.com/jobs/AJ46406 のデータ取得完了
✅ https://www.centrepeople.com/jobs/MY46358 のデータ取得完了
✅ https://www.centrepeople.com/jobs/LS46385 のデータ取得完了
✅ https://www.centrepeople.com/jobs/TY46330 のデータ取得完了
✅ https://www.centrepeople.com/jobs/TK46369 のデータ取得完了
✅ https://www.centrepeople.com/jobs/WL46395 のデータ取得完了
✅ https://www.centrepeople.com/jobs/KM46404 のデータ取得完了
✅ https://www.centrepeople.com/jobs/KK46403 のデータ取得完了
✅ https://www.centrepeople.com/jobs/HN46282 のデータ取得完了
✅ https://www.centrepeople.com/jobs/SF46397 のデータ取得完了
✅ https://www.centrepeople.com/jobs/KM46398 のデータ取得完了
✅ https://www.centrepeople.com/jobs/KM46389 のデータ取得完了
✅

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# MixB - Extract URL

In [ ]:
import requests
import csv
from lxml import html
from datetime import datetime

# ページ数の範囲 (5ページ分)
page_range = range(1, 6)  # j = 1 ~ 5

# ベースURL
base_url = 'https://uk.mixb.net'

# 日付と時間を取得してファイル名に追加
current_time = datetime.now().strftime('%Y%m%d-%H%M')
csv_file_name = f'mixb_job_links_{current_time}.csv'

# CSVに書き込む準備
with open(csv_file_name, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Job URL'])  # ヘッダー行を書く

    # 各ページを処理
    for j in page_range:
        url = f'{base_url}/job?page={j}'

        # j = 1のときはx = 4、それ以外はx = 3
        x = 4 if j == 1 else 3

        # Webページを取得
        response = requests.get(url)
        tree = html.fromstring(response.content)

        # i = 2 ~ 33 の範囲のXPath部分を抽出
        for i in range(2, 34):  # iは2から33まで
            xpath = f'/html/body/div[2]/main/section[{x}]/div/ul/li[{i}]/a'
            link = tree.xpath(xpath)
            if link:
                # ベースURLとリンクを結合
                full_link = base_url + link[0].get('href')
                writer.writerow([full_link])  # CSVにリンクを追加

# 結果を保存したCSVファイルへのリンク
print(f'CSVファイルが保存されました: {csv_file_name}')


CSVファイルが保存されました: mixb_job_links_20251120-1400.csv


# MixB - Extract Job Detail

In [ ]:
import requests
import pandas as pd
from lxml import html
from google.colab import files  # Google Colab用のファイルアップロード
from IPython.display import display  # 表示用
from datetime import datetime  # 現在時刻取得用

# CSVファイルをアップロード
uploaded = files.upload()

# CSVファイル名を取得
csv_filename = list(uploaded.keys())[0]

# CSVファイルを読み込んでURLリストを取得（カラム名を確認）
df_input = pd.read_csv(csv_filename)
print(df_input.head())  # 確認用に先頭の5行を表示

# "Job URL" カラムがない場合、"URL" などの代替カラムを試す
if "Job URL" in df_input.columns:
    urls = df_input["Job URL"].dropna().tolist()
elif "URL" in df_input.columns:
    urls = df_input["URL"].dropna().tolist()
else:
    raise ValueError("CSVファイルに 'Job URL' または 'URL' のカラムが見つかりません。")

# 判定する項目
target_labels = [
    '業種', '職種', '社名', '必要な語学力', '雇用形態', 'ビザ',
    '担当者', '電話番号', 'メールアドレス', 'URL', 'エリア名', '最寄駅'
]

# "詳細"部分を抽出するXPath
xpath_detail = "//div[@class='p-articles-detail']"

# 結果を格納するリスト
all_data = []

# ヘッダー情報（スクレイピング対策回避用）
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
}

# 各URLに対してデータを抽出
for url in urls:
    try:
        response = requests.get(url, headers=headers, timeout=10)
        tree = html.fromstring(response.content)

        # データ結果の初期化
        result = {label: 'N/A' for label in target_labels}

        # ラベルとデータのペアをマッチング
        for i in range(1, 8):
            label_xpath = f'/html/body/div[4]/main/section[2]/div[2]/div/div[1]/div/div[{i}]/div[1]/text()'
            data_xpath = f'/html/body/div[4]/main/section[2]/div[2]/div/div[1]/div/div[{i}]/div[2]/text()'

            label = tree.xpath(label_xpath)
            data_value = tree.xpath(data_xpath)

            if label and label[0].strip() in target_labels:
                result[label[0].strip()] = data_value[0].strip() if data_value else 'N/A'

        for j in range(1, 4):
            label_xpath = f'/html/body/div[4]/main/section[2]/div[2]/div/div[2]/div/div[{j}]/div[1]/text()'
            data_xpath = f'/html/body/div[4]/main/section[2]/div[2]/div/div[2]/div/div[{j}]/div[2]/text()'

            label = tree.xpath(label_xpath)
            data_value = tree.xpath(data_xpath)

            if label and label[0].strip() in target_labels:
                result[label[0].strip()] = data_value[0].strip() if data_value else 'N/A'

        # "詳細"部分を抽出
        detail_section = tree.xpath(xpath_detail)
        result['詳細'] = detail_section[0].text_content().strip() if detail_section else 'N/A'

        # URLを追加
        result['URL'] = url

        # データをall_dataリストに追加
        all_data.append(result)

    except Exception as e:
        print(f"Error processing {url}: {e}")

# DataFrameを作成
df = pd.DataFrame(all_data)

# 現在時刻を取得してCSVファイル名を作成
current_time = datetime.now().strftime('%Y%m%d-%H%M')
csv_file_name = f'mixb_job_details_{current_time}.csv'

# CSVファイルとして保存
df.to_csv(csv_file_name, index=False)

# CSVファイルをダウンロード
files.download(csv_file_name)

# 最初の5行を表示
display(df.head())

print(f"データ収集完了。保存ファイル: {csv_file_name}")


Saving mixb_job_links_20251120-1400.csv to mixb_job_links_20251120-1400 (1).csv
                                   Job URL
0  https://uk.mixb.net/job/articles/295900
1  https://uk.mixb.net/job/articles/286929
2  https://uk.mixb.net/job/articles/295894
3  https://uk.mixb.net/job/articles/263973
4  https://uk.mixb.net/job/articles/294735


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,業種,職種,社名,必要な語学力,雇用形態,ビザ,担当者,電話番号,メールアドレス,URL,エリア名,最寄駅,詳細
0,広告・企画・デザイン,,N/A,不問,,,西山,N/A,,https://uk.mixb.net/job/articles/295900,"Camden\n,\nall",N/A,当社では、WEB求人サイトに広告を掲載するお仕事をお任せいたします。\n\n【お仕事内容】\...
1,飲食・レストラン,,Cocoro Group,仕事である程度使える,,N/A,リクルート担当,N/A,,https://uk.mixb.net/job/articles/286929,"Haringey\n,\nHighgate",Highgate,広告をご覧いただき、ありがとうございます！\n\nCocoro Group は、現在ロンドン...
2,広告・企画・デザイン,,N/A,不問,,,小林,N/A,,https://uk.mixb.net/job/articles/295894,"Bromley\n,\nall",N/A,当社では、WEB求人サイトに広告を掲載するお仕事をお任せいたします。\n\n【お仕事内容】\...
3,飲食・レストラン,,MUGEN,簡単な意思の疎通ができる,,,Nagano,N/A,N/A,https://uk.mixb.net/job/articles/263973,"City\n,\nChancery lane, Farringdon, Holborn",Chancery Lane,City近くのMugenレストランでは、昼夜働けるホールスタッフを募集しています！\n飲食業...
4,飲食・レストラン,,N/A,ある程度の意思の疎通ができる,,,Shenali,N/A,,https://uk.mixb.net/job/articles/294735,"ロンドンエリア外\n,\nBirmingham",N/A,イベント会場にて日本食の接客、販売、調理ができる方を募集しています！\n\n11月28〜30...


データ収集完了。保存ファイル: mixb_job_details_20251120-1405.csv


In [ ]:
import requests
import pandas as pd
from lxml import html
from google.colab import files  # Google Colab用のファイルアップロード
from IPython.display import display  # 表示用
from datetime import datetime  # 現在時刻取得用

# CSVファイルをアップロード
uploaded = files.upload()

# CSVファイル名を取得
csv_filename = list(uploaded.keys())[0]

# CSVファイルを読み込んでURLリストを取得（カラム名を確認）
df_input = pd.read_csv(csv_filename)
print(df_input.head())  # 確認用に先頭の5行を表示

# "Job URL" カラムがない場合、"URL" などの代替カラムを試す
if "Job URL" in df_input.columns:
    urls = df_input["Job URL"].dropna().tolist()
elif "URL" in df_input.columns:
    urls = df_input["URL"].dropna().tolist()
else:
    raise ValueError("CSVファイルに 'Job URL' または 'URL' のカラムが見つかりません。")

# 判定する項目（←★ 日付を追加）
target_labels = [
    '業種', '職種', '社名', '必要な語学力', '雇用形態', 'ビザ',
    '担当者', '電話番号', 'メールアドレス', 'URL', 'エリア名', '最寄駅',
    '日付'
]

# "詳細"部分を抽出するXPath
xpath_detail = "//div[@class='p-articles-detail']"

# 日付のXPath（←★ 追加）
xpath_date = "/html/body/div[4]/main/section[2]/div[1]/div/span/text()"

# 結果を格納するリスト
all_data = []

# ヘッダー情報（スクレイピング対策回避用）
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
}

# 各URLに対してデータを抽出
for url in urls:
    try:
        response = requests.get(url, headers=headers, timeout=10)
        tree = html.fromstring(response.content)

        # データ結果の初期化
        result = {label: 'N/A' for label in target_labels}

        # ラベルとデータのペアをマッチング
        for i in range(1, 8):
            label_xpath = f'/html/body/div[4]/main/section[2]/div[2]/div/div[1]/div/div[{i}]/div[1]/text()'
            data_xpath = f'/html/body/div[4]/main/section[2]/div[2]/div/div[1]/div/div[{i}]/div[2]/text()'

            label = tree.xpath(label_xpath)
            data_value = tree.xpath(data_xpath)

            if label and label[0].strip() in target_labels:
                result[label[0].strip()] = data_value[0].strip() if data_value else 'N/A'

        for j in range(1, 4):
            label_xpath = f'/html/body/div[4]/main/section[2]/div[2]/div/div[2]/div/div[{j}]/div[1]/text()'
            data_xpath = f'/html/body/div[4]/main/section[2]/div[2]/div/div[2]/div/div[{j}]/div[2]/text()'

            label = tree.xpath(label_xpath)
            data_value = tree.xpath(data_xpath)

            if label and label[0].strip() in target_labels:
                result[label[0].strip()] = data_value[0].strip() if data_value else 'N/A'

        # ★ 日付を抽出して追加
        date_value = tree.xpath(xpath_date)
        result['日付'] = date_value[0].strip() if date_value else 'N/A'

        # "詳細"部分を抽出
        detail_section = tree.xpath(xpath_detail)
        result['詳細'] = detail_section[0].text_content().strip() if detail_section else 'N/A'

        # URLを追加
        result['URL'] = url

        # データをall_dataリストに追加
        all_data.append(result)

    except Exception as e:
        print(f"Error processing {url}: {e}")

# DataFrameを作成
df = pd.DataFrame(all_data)

# 現在時刻を取得してCSVファイル名を作成
current_time = datetime.now().strftime('%Y%m%d-%H%M')
csv_file_name = f'mixb_job_details_{current_time}.csv'

# CSVファイルとして保存
df.to_csv(csv_file_name, index=False)

# CSVファイルをダウンロード
files.download(csv_file_name)

# 最初の5行を表示
display(df.head())

print(f"データ収集完了。保存ファイル: {csv_file_name}")


Saving mixb_job_links_20251120-1400.csv to mixb_job_links_20251120-1400.csv
                                   Job URL
0  https://uk.mixb.net/job/articles/295900
1  https://uk.mixb.net/job/articles/286929
2  https://uk.mixb.net/job/articles/295894
3  https://uk.mixb.net/job/articles/263973
4  https://uk.mixb.net/job/articles/294735


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,業種,職種,社名,必要な語学力,雇用形態,ビザ,担当者,電話番号,メールアドレス,URL,エリア名,最寄駅,日付,詳細
0,広告・企画・デザイン,,N/A,不問,,,西山,N/A,,https://uk.mixb.net/job/articles/295900,"Camden\n,\nall",N/A,2025/11/20 13:56:03,当社では、WEB求人サイトに広告を掲載するお仕事をお任せいたします。\n\n【お仕事内容】\...
1,飲食・レストラン,,Cocoro Group,仕事である程度使える,,N/A,リクルート担当,N/A,,https://uk.mixb.net/job/articles/286929,"Haringey\n,\nHighgate",Highgate,2025/11/20 13:51:37,広告をご覧いただき、ありがとうございます！\n\nCocoro Group は、現在ロンドン...
2,広告・企画・デザイン,,N/A,不問,,,小林,N/A,,https://uk.mixb.net/job/articles/295894,"Bromley\n,\nall",N/A,2025/11/20 13:41:12,当社では、WEB求人サイトに広告を掲載するお仕事をお任せいたします。\n\n【お仕事内容】\...
3,飲食・レストラン,,MUGEN,簡単な意思の疎通ができる,,,Nagano,N/A,N/A,https://uk.mixb.net/job/articles/263973,"City\n,\nChancery lane, Farringdon, Holborn",Chancery Lane,2025/11/20 13:37:43,City近くのMugenレストランでは、昼夜働けるホールスタッフを募集しています！\n飲食業...
4,飲食・レストラン,,N/A,ある程度の意思の疎通ができる,,,Shenali,N/A,,https://uk.mixb.net/job/articles/294735,"ロンドンエリア外\n,\nBirmingham",N/A,2025/11/20 12:41:53,イベント会場にて日本食の接客、販売、調理ができる方を募集しています！\n\n11月28〜30...


データ収集完了。保存ファイル: mixb_job_details_20251120-2351.csv


In [ ]:
!pip install gspread oauth2client

import requests
import pandas as pd
from lxml import html
from google.colab import files
from IPython.display import display
from datetime import datetime

import gspread
from google.auth import default

# ▼ Google 認証
creds, _ = default()
gc = gspread.authorize(creds)


# ▼ CSVファイルをアップロード
uploaded = files.upload()

# ファイル名を取得
csv_filename = list(uploaded.keys())[0]

# CSVを読み込み
df_input = pd.read_csv(csv_filename)
print(df_input.head())

# URLカラム取得
if "Job URL" in df_input.columns:
    urls = df_input["Job URL"].dropna().tolist()
elif "URL" in df_input.columns:
    urls = df_input["URL"].dropna().tolist()
else:
    raise ValueError("CSVファイルに 'Job URL' または 'URL' カラムがありません。")

# 取得するラベル
target_labels = [
    '業種', '職種', '社名', '必要な語学力', '雇用形態', 'ビザ',
    '担当者', '電話番号', 'メールアドレス', 'URL', 'エリア名', '最寄駅',
    '日付'
]

xpath_detail = "//div[@class='p-articles-detail']"
xpath_date = "/html/body/div[4]/main/section[2]/div[1]/div/span/text()"

all_data = []

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
}

# ▼ スクレイピング処理
for url in urls:
    try:
        response = requests.get(url, headers=headers, timeout=10)
        tree = html.fromstring(response.content)

        result = {label: 'N/A' for label in target_labels}

        # 左側の項目（業種 〜 最寄駅）
        for i in range(1, 8):
            label_xpath = f'/html/body/div[4]/main/section[2]/div[2]/div/div[1]/div/div[{i}]/div[1]/text()'
            data_xpath = f'/html/body/div[4]/main/section[2]/div[2]/div/div[1]/div/div[{i}]/div[2]/text()'

            label = tree.xpath(label_xpath)
            data_value = tree.xpath(data_xpath)

            if label and label[0].strip() in target_labels:
                result[label[0].strip()] = data_value[0].strip() if data_value else 'N/A'

        # 右側の項目（担当者 〜 最寄駅）
        for j in range(1, 4):
            label_xpath = f'/html/body/div[4]/main/section[2]/div[2]/div/div[2]/div/div[{j}]/div[1]/text()'
            data_xpath = f'/html/body/div[4]/main/section[2]/div[2]/div/div[2]/div/div[{j}]/div[2]/text()'

            label = tree.xpath(label_xpath)
            data_value = tree.xpath(data_xpath)

            if label and label[0].strip() in target_labels:
                result[label[0].strip()] = data_value[0].strip() if data_value else 'N/A'

        # ▼ 日付の追加
        date_value = tree.xpath(xpath_date)
        result['日付'] = date_value[0].strip() if date_value else 'N/A'

        # ▼ 詳細抽出
        detail_section = tree.xpath(xpath_detail)
        result['詳細'] = detail_section[0].text_content().strip() if detail_section else 'N/A'

        # URL追加
        result['URL'] = url

        all_data.append(result)

    except Exception as e:
        print(f"Error processing {url}: {e}")

# ▼ DataFrame 化
df = pd.DataFrame(all_data)
display(df.head())

# ▼ 現在時刻
current_time = datetime.now().strftime('%Y%m%d-%H%M')

# ▼ 既存のスプレッドシート ID
spreadsheet_id = "1qYmUrEHDE1vgyb13U4NytzTlbY9AKGWzuqDx0--6FWY"

# ▼ シートを開く
sh = gc.open_by_key(spreadsheet_id)

# ▼ 新しいシート名（例：mixb_20250101-2359）
new_sheet_name = f"mixb_{current_time}"

# ▼ シート追加（行2000・列50）
worksheet = sh.add_worksheet(title=new_sheet_name, rows=2000, cols=50)

# ▼ DataFrame を書き込む
worksheet.update([df.columns.values.tolist()] + df.values.tolist())

print("既存 Google Spreadsheet に新しいシートを追加して書き込み完了！")
print(f"追加したシート名: {new_sheet_name}")
print(f"URL: https://docs.google.com/spreadsheets/d/{spreadsheet_id}")


Saving mixb_job_links_20251120-1400.csv to mixb_job_links_20251120-1400 (1).csv
                                   Job URL
0  https://uk.mixb.net/job/articles/295900
1  https://uk.mixb.net/job/articles/286929
2  https://uk.mixb.net/job/articles/295894
3  https://uk.mixb.net/job/articles/263973
4  https://uk.mixb.net/job/articles/294735


,業種,職種,社名,必要な語学力,雇用形態,ビザ,担当者,電話番号,メールアドレス,URL,エリア名,最寄駅,日付,詳細
0,広告・企画・デザイン,,N/A,不問,,,西山,N/A,,https://uk.mixb.net/job/articles/295900,"Camden\n,\nall",N/A,2025/11/20 13:56:03,当社では、WEB求人サイトに広告を掲載するお仕事をお任せいたします。\n\n【お仕事内容】\...
1,飲食・レストラン,,Cocoro Group,仕事である程度使える,,N/A,リクルート担当,N/A,,https://uk.mixb.net/job/articles/286929,"Haringey\n,\nHighgate",Highgate,2025/11/20 13:51:37,広告をご覧いただき、ありがとうございます！\n\nCocoro Group は、現在ロンドン...
2,広告・企画・デザイン,,N/A,不問,,,小林,N/A,,https://uk.mixb.net/job/articles/295894,"Bromley\n,\nall",N/A,2025/11/20 13:41:12,当社では、WEB求人サイトに広告を掲載するお仕事をお任せいたします。\n\n【お仕事内容】\...
3,飲食・レストラン,,MUGEN,簡単な意思の疎通ができる,,,Nagano,N/A,N/A,https://uk.mixb.net/job/articles/263973,"City\n,\nChancery lane, Farringdon, Holborn",Chancery Lane,2025/11/20 13:37:43,City近くのMugenレストランでは、昼夜働けるホールスタッフを募集しています！\n飲食業...
4,飲食・レストラン,,N/A,ある程度の意思の疎通ができる,,,Shenali,N/A,,https://uk.mixb.net/job/articles/294735,"ロンドンエリア外\n,\nBirmingham",N/A,2025/11/20 12:41:53,イベント会場にて日本食の接客、販売、調理ができる方を募集しています！\n\n11月28〜30...


既存 Google Spreadsheet に新しいシートを追加して書き込み完了！
追加したシート名: mixb_20251121-0014
URL: https://docs.google.com/spreadsheets/d/1qYmUrEHDE1vgyb13U4NytzTlbY9AKGWzuqDx0--6FWY


# Access Appointments - Job Links Scraping

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
from google.colab import files  # Colab用のファイルダウンロード機能
from datetime import datetime  # 日付と時間を取得

# ベースURL
base_url = "https://www.access-appointments.com/page/{}/?catnum&placenum&s&submit=Search"

# ヘッダー情報（スクレイピング対策回避用）
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
}

# 取得したURLを格納するリスト
job_links = []

# j = 1 ~ 10 のページをループ
for j in range(1, 11):
    url = base_url.format(j)
    response = requests.get(url, headers=headers)

    # ステータスコードが200（成功）の場合のみ処理
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")

        # i = 1 ~ 10 の各求人情報のURLを取得
        for i in range(1, 11):
            job_element = soup.select_one(f"main > div:nth-of-type({i}) h2 a")
            if job_element and job_element.get("href"):
                job_links.append(job_element["href"])

    else:
        print(f"Failed to fetch page {j}: {response.status_code}")

# 現在の日付と時間を取得（yyyymmdd-hhmm形式）
current_time = datetime.now().strftime('%Y%m%d-%H%M')

# 新しいCSVファイル名を作成
csv_filename = f"access_appointments_job_links_{current_time}.csv"

# CSVファイルに書き出し
with open(csv_filename, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Job URL"])  # ヘッダー行
    for link in job_links:
        writer.writerow([link])

# Google Colab から CSV をダウンロード
files.download(csv_filename)

# 完了メッセージ
print(f"\nCSVファイル '{csv_filename}' をダウンロードしました！")
print(f"Total job links collected: {len(job_links)}")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


CSVファイル 'access_appointments_job_links_20250323-1039.csv' をダウンロードしました！
Total job links collected: 98


# Access Appointment - Job Details Scraping

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup, Comment
from google.colab import files
from datetime import datetime  # 日付と時間を取得

# アップロードされたCSVファイルを読み込む
uploaded = files.upload()

# CSVファイル名を指定
csv_filename = list(uploaded.keys())[0]

# CSVファイルを読み込んでURLリストを取得
urls = pd.read_csv(csv_filename)["Job URL"].tolist()  # CSV内のURL列を指定

# 結果を格納するリスト
results = []

# ヘッダー情報（スクレイピング対策回避用）
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
}

# 各URLに対して情報をスクレイピング
for url in urls:
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")

        # 1. Category（カテゴリ）
        category_element = soup.select_one("article header div div h2 a")
        category = category_element.text.strip() if category_element else "N/A"

        # 2. Job-Title（求人タイトル）
        title_element = soup.select_one("article header div div h3")
        job_title = title_element.text.strip() if title_element else "N/A"

        # 3. Location-Salary-RefNo（勤務地・給与・求人番号）
        location_salary_ref_element = soup.select_one("article div p")
        location_salary_ref = location_salary_ref_element.text.strip() if location_salary_ref_element else "N/A"

        # 4. All-Content（不要な部分を削除）
        content_element = soup.select_one("article > div:nth-of-type(1)")

        if content_element:
            # 最初の p タグ（給与情報など）を削除
            first_p = content_element.find("p")
            if first_p:
                first_p.extract()

            # コメントタグ以降の不要部分を削除
            for comment in content_element.find_all(string=lambda text: isinstance(text, Comment)):
                parent = comment.find_parent()
                while parent:
                    next_sibling = parent.find_next_sibling()
                    while next_sibling:
                        next_sibling.extract()  # コメント以降の部分を削除
                        next_sibling = parent.find_next_sibling()
                    parent = parent.find_parent()

            # フォームや応募関連の要素を削除
            for unwanted in content_element.find_all(["form", "input", "textarea", "button", "label", "select", "option", "script", "noscript", "iframe"]):
                unwanted.decompose()

            # 最後の div 要素を削除（余計な情報が入りやすい部分）
            div_elements = content_element.find_all("div")
            if div_elements:
                div_elements[-1].decompose()

            # 綺麗にテキストを取得
            all_content = content_element.get_text("\n", strip=True)
        else:
            all_content = "N/A"

        # 結果をリストに追加
        results.append({
            "URL": url,
            "Category": category,
            "Job-Title": job_title,
            "Location-Salary-RefNo": location_salary_ref,
            "All-Content": all_content
        })

    else:
        # ステータスコードが200ではない場合はエラーメッセージ
        results.append({
            "URL": url,
            "Category": "N/A",
            "Job-Title": "N/A",
            "Location-Salary-RefNo": "N/A",
            "All-Content": "N/A"
        })

# 結果をDataFrameに変換
df_results = pd.DataFrame(results)

# 現在の日付と時間を取得（yyyymmdd-hhmm形式）
current_time = datetime.now().strftime('%Y%m%d-%H%M')

# 新しいCSVファイル名を作成
output_csv_filename = f"access_appointments_job_details_{current_time}.csv"

# CSVファイルとして保存
df_results.to_csv(output_csv_filename, index=False)

# Google ColabからCSVファイルをダウンロード
files.download(output_csv_filename)


Saving access_appointments_job_links_20250323-1039.csv to access_appointments_job_links_20250323-1039 (1).csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# JAC Recruitment - Job Links Scraping

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
from datetime import datetime

# 求人情報のURLを保存するリスト
job_links = []

# 1ページ目から5ページ目までループ
base_url = "https://www.jac-recruitment.co.uk"
for j in range(1, 6):
    url = f"{base_url}/jobs?page={j}"

    # ページのHTMLを取得
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # 求人情報のリンクを抽出
    for i in range(1, 21):
        link_element = soup.select_one(f'#body > section > div:nth-of-type(2) > div:nth-of-type(3) > ul > li:nth-of-type({i}) > div > div:nth-of-type(1) > div:nth-of-type(2) > div:nth-of-type(1) > a')
        if link_element:
            full_link = base_url + link_element['href']  # ベースURLを前に追加
            job_links.append(full_link)

# 現在の日付と時刻を "yyyymmdd-hhmm" の形式で取得
timestamp = datetime.now().strftime("%Y%m%d-%H%M")

# CSVファイル名を指定
output_filename = f"jac_recruitment_job_links_{timestamp}.csv"

# 結果をCSVファイルに保存
with open(output_filename, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Job URL"])  # ヘッダー
    for link in job_links:
        writer.writerow([link])

print(f"CSVファイル '{output_filename}' に保存しました。")


CSVファイル 'jac_recruitment_job_links_20250323-1039.csv' に保存しました。


# JAC Recruitment - Job Details Scraping

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from google.colab import files
from datetime import datetime

# CSVファイルをアップロード
uploaded = files.upload()

# CSVファイル名を指定
csv_filename = list(uploaded.keys())[0]

# CSVファイルを読み込んでURLリストを取得
urls = pd.read_csv(csv_filename)["Job URL"].tolist()  # CSV内のURL列を指定

# 結果を格納するリスト
results = []

# ヘッダー情報（スクレイピング対策回避用）
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
}

# 各URLに対してデータを取得
for url in urls:
    job_data = {
        "URL": url,
        "Job Title": "N/A",
        "Contract Type": "N/A",
        "Location": "N/A",
        "Specialisation": "N/A",
        "Salary": "N/A",  # 追加
        "Reference": "N/A",
        "Job Published": "N/A",
        "Work Arrangement": "N/A",
        "Job Description": "N/A"
    }

    # URLの取得とデータ抽出
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")

        # テーブルデータの抽出 (1〜8) → 1〜7 だったのを 1〜8 に変更
        for i in range(1, 9):
            key_element = soup.select_one(f'#primary-content > div:nth-of-type(2) table tbody tr:nth-of-type({i}) td:nth-of-type(1)')
            value_element = soup.select_one(f'#primary-content > div:nth-of-type(2) table tbody tr:nth-of-type({i}) td:nth-of-type(2)')

            if key_element and value_element:
                key_text = key_element.get_text(strip=True)
                value_text = value_element.get_text(strip=True)

                if key_text == "Job title:":
                    job_data["Job Title"] = value_text
                elif key_text == "Contract type:":
                    job_data["Contract Type"] = value_text
                elif key_text == "Location:":
                    job_data["Location"] = value_text
                elif key_text == "Specialisation:":
                    job_data["Specialisation"] = value_text
                elif key_text == "Salary:":  # 追加
                    job_data["Salary"] = value_text
                elif key_text == "Job published:":
                    job_data["Job Published"] = value_text
                elif key_text == "Work Arrangement:":
                    job_data["Work Arrangement"] = value_text

        # Reference の取得
        ref_element = soup.select_one('#reference')
        job_data["Reference"] = ref_element.get_text(strip=True) if ref_element else "N/A"

        # Job Description の取得
        desc_element = soup.select_one('#primary-content > article')
        job_data["Job Description"] = desc_element.get_text("\n", strip=True) if desc_element else "N/A"

    # 結果リストに追加
    results.append(job_data)

# 結果をデータフレームに変換
df = pd.DataFrame(results)

# 現在の日付と時刻を "yyyymmdd-hhmm" の形式で取得
timestamp = datetime.now().strftime("%Y%m%d-%H%M")

# CSVファイル名を指定
output_filename = f"jac_recruitment_job_details_{timestamp}.csv"

# CSVファイルとして保存
df.to_csv(output_filename, index=False)

# CSVファイルをダウンロード
files.download(output_filename)


Saving jac_recruitment_job_links_20250323-1039.csv to jac_recruitment_job_links_20250323-1039 (2).csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# People First - Job List Scraping

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
from google.colab import files

# 現在の日付と時刻を取得し、ファイル名に適用
timestamp = datetime.now().strftime("%Y%m%d-%H%M")
filename = f"people_first_job_links_{timestamp}.csv"

# ベースURL
base_url = "https://www.people-first.co.uk/jobs?departments[]=1&page={}"

# ヘッダー設定（アクセスブロック対策）
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# 求人URLリスト
all_job_links = []

# 1~4ページをループで取得
for y in range(1, 5):  # {y} = 1 ~ 4
    url = base_url.format(y)
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    # 各ページの求人URLを取得
    for i in range(1, 13):  # x = 1 ~ 12
        job_element = soup.select_one(f"body > div:nth-of-type(1) main section div article div:nth-of-type(1) ul li:nth-of-type({i}) a")
        if job_element:
            job_url = "https://www.people-first.co.uk" + job_element["href"]
            all_job_links.append(job_url)

# DataFrameに変換し、CSVに保存
df = pd.DataFrame(all_job_links, columns=["Job URL"])
df.to_csv(filename, index=False)

# ファイルを自動ダウンロード
files.download(filename)

print(f"CSVファイル '{filename}' が作成され、ダウンロードが開始されました！")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

CSVファイル 'people_first_job_links_20250327-1451.csv' が作成され、ダウンロードが開始されました！
